In [72]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [36]:
train_data=pd.read_csv('Features/train_data.csv',index_col=0)
test_data=pd.read_csv('Features/test_data.csv',index_col=0)

In [37]:
train_data.head()

,id,keyword,location,text,target,caracteres_usados,menciones_realizadas,permite_location,use_keyword,cita_url,use_hashtag,cant_palabras,cant_abreviaciones,location_usa,has_emoji,has_repeated_letter
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,0,0,0,0,1,13,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,0,0,0,0,0,7,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,0,0,0,0,0,22,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,0,0,0,0,1,9,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,0,0,0,0,2,17,0,0,0,0


## Tokenize

In [38]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [127]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [40]:
tokenized = train_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

## Padding

In [41]:
lenghts = []
for tweet in tokenized:
    lenghts.append(len(tweet))
    
max(lenghts)

84

In [46]:
def hacer_padding(data, n):
    for lista in data:
        long = n - len(lista)
        for i in range(long):
            lista.append(0)
    return data

In [123]:
padded = hacer_padding(tokenized, max(lenghts))

## Procesamiento con distilBERT

In [125]:
input_ids = torch.tensor(padded)

In [63]:
with torch.no_grad():
    last_hidden_states = model(input_ids)

In [64]:
features = last_hidden_states[0][:,0,:].numpy()

## Creo el modelo y evaluo

In [65]:
labels = train_data['target']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [135]:
lr_clf = LogisticRegression(max_iter=1000)
bert_model = lr_clf.fit(train_features, train_labels)

In [136]:
lr_clf.score(test_features, test_labels)

0.8203781512605042

In [137]:
Y_pred = bert_model.predict(test_features)

In [138]:
print("Accuracy:", metrics.accuracy_score(test_labels, Y_pred))

Accuracy: 0.8203781512605042


## Proceso el Test

In [128]:
tokenized_t = test_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [129]:
lenghts_t = []
for tweet in tokenized_t:
    lenghts_t.append(len(tweet))
    
max(lenghts_t)

73

In [130]:
padded_t = hacer_padding(tokenized_t, max(lenghts_t))

In [131]:
input_ids_t = torch.tensor(padded_t)

In [132]:
with torch.no_grad():
    last_hidden_states_t = model(input_ids_t)

In [133]:
features_t = last_hidden_states_t[0][:,0,:].numpy()

In [152]:
test_data['prediction'] = bert_model.predict(features_t)

In [153]:
submit_kaggle = pd.DataFrame({'id':test_data['id'],'target':test_data['prediction']})

In [154]:
submit_kaggle['target'] = submit_kaggle['target'].transform(lambda x: 0 if x < 0.5 else 1)

In [155]:
submit_kaggle.to_csv('submit_kaggle_BERT.csv',index=False)

In [156]:
submit_kaggle

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
